# Dėmesio mechanizmai ir transformatoriai

Viena iš pagrindinių pasikartojančių tinklų trūkumų yra ta, kad visi sekos žodžiai turi vienodą įtaką rezultatui. Tai lemia neoptimalų veikimą naudojant standartinius LSTM koduotojo-dekoduotojo modelius sekų užduotims, tokioms kaip pavadintų objektų atpažinimas ar mašininis vertimas. Iš tikrųjų tam tikri žodžiai įvesties sekoje dažnai turi didesnę įtaką išvesties sekai nei kiti.

Apsvarstykime sekos į seką modelį, pavyzdžiui, mašininį vertimą. Jis įgyvendinamas naudojant du pasikartojančius tinklus, kur vienas tinklas (**koduotojas**) suspaudžia įvesties seką į paslėptą būseną, o kitas tinklas (**dekoduotojas**) išskleidžia šią paslėptą būseną į išverstą rezultatą. Problema su šiuo metodu yra ta, kad tinklo galutinė būsena sunkiai prisimena sakinio pradžią, todėl modelis prastai veikia su ilgais sakiniais.

**Dėmesio mechanizmai** suteikia galimybę įvertinti kiekvieno įvesties vektoriaus kontekstinę įtaką kiekvienai RNN išvesties prognozei. Tai įgyvendinama sukuriant trumpesnius ryšius tarp įvesties RNN tarpinių būsenų ir išvesties RNN. Tokiu būdu, generuojant išvesties simbolį $y_t$, atsižvelgiama į visas įvesties paslėptas būsenas $h_i$, su skirtingais svorio koeficientais $\alpha_{t,i}$.

![Vaizdas, rodantis koduotojo/dekoduotojo modelį su adityviniu dėmesio sluoksniu](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Koduotojo-dekoduotojo modelis su adityviniu dėmesio mechanizmu iš [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cituota iš [šio tinklaraščio įrašo](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Dėmesio matrica $\{\alpha_{i,j}\}$ atspindi, kokiu mastu tam tikri įvesties žodžiai prisideda prie tam tikro žodžio generavimo išvesties sekoje. Žemiau pateiktas tokios matricos pavyzdys:

![Vaizdas, rodantis pavyzdinį suderinimą, rastą naudojant RNNsearch-50, paimta iš Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Paveikslas paimtas iš [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (3 pav.)*

Dėmesio mechanizmai yra atsakingi už daugelį dabartinių ar beveik dabartinių pažangiausių rezultatų natūralios kalbos apdorojime. Tačiau dėmesio pridėjimas žymiai padidina modelio parametrų skaičių, o tai sukėlė mastelio problemas su RNN. Vienas iš pagrindinių RNN mastelio apribojimų yra tai, kad modelių pasikartojantis pobūdis apsunkina mokymo partijų kūrimą ir lygiagretinimą. RNN kiekvienas sekos elementas turi būti apdorojamas nuosekliai, todėl jo negalima lengvai lygiagrečiai apdoroti.

Dėmesio mechanizmų pritaikymas kartu su šiuo apribojimu paskatino sukurti dabartinius pažangiausius transformatorių modelius, kuriuos šiandien naudojame, tokius kaip BERT ar OpenGPT3.

## Transformatorių modeliai

Užuot perdavę kiekvienos ankstesnės prognozės kontekstą į kitą vertinimo žingsnį, **transformatorių modeliai** naudoja **pozicinius kodavimus** ir dėmesį, kad užfiksuotų įvesties kontekstą tam tikrame teksto lange. Žemiau pateiktas vaizdas rodo, kaip poziciniai kodavimai su dėmesiu gali užfiksuoti kontekstą tam tikrame lange.

![Animuotas GIF, rodantis, kaip atliekami vertinimai transformatorių modeliuose.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Kadangi kiekviena įvesties pozicija yra nepriklausomai susieta su kiekviena išvesties pozicija, transformatoriai gali geriau lygiagrečiai apdoroti nei RNN, o tai leidžia kurti daug didesnius ir išraiškingesnius kalbos modelius. Kiekviena dėmesio galvutė gali būti naudojama mokytis skirtingų žodžių tarpusavio ryšių, kurie pagerina natūralios kalbos apdorojimo užduotis.

**BERT** (Bidirectional Encoder Representations from Transformers) yra labai didelis daugiasluoksnis transformatorių tinklas su 12 sluoksnių *BERT-base* versijoje ir 24 sluoksniais *BERT-large* versijoje. Modelis pirmiausia yra iš anksto apmokomas naudojant didelį tekstų korpusą (Vikipedija + knygos) taikant nesupervizuotą mokymąsi (prognozuojant užmaskuotus žodžius sakinyje). Išankstinio mokymo metu modelis įgyja reikšmingą kalbos supratimo lygį, kurį vėliau galima pritaikyti kitoms duomenų aibėms naudojant smulkųjį derinimą. Šis procesas vadinamas **perkėlimo mokymusi**.

![Paveikslas iš http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Yra daug transformatorių architektūrų variantų, įskaitant BERT, DistilBERT, BigBird, OpenGPT3 ir daugiau, kuriuos galima smulkiai derinti. [HuggingFace paketas](https://github.com/huggingface/) suteikia saugyklą daugeliui šių architektūrų mokyti naudojant PyTorch.

## BERT naudojimas teksto klasifikavimui

Pažiūrėkime, kaip galime naudoti iš anksto apmokytą BERT modelį, kad išspręstume tradicinę užduotį: sekos klasifikavimą. Mes klasifikuosime savo pradinį AG News duomenų rinkinį.

Pirmiausia įkelkime HuggingFace biblioteką ir mūsų duomenų rinkinį:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Kadangi naudosime iš anksto apmokytą BERT modelį, mums reikės naudoti specifinį tokenizatorių. Pirmiausia įkelsime tokenizatorių, susietą su iš anksto apmokytu BERT modeliu.

HuggingFace biblioteka turi iš anksto apmokytų modelių saugyklą, kurią galite naudoti tiesiog nurodydami jų pavadinimus kaip argumentus `from_pretrained` funkcijoms. Visi reikalingi dvejetainiai modelio failai bus automatiškai atsisiųsti.

Tačiau tam tikrais atvejais jums gali prireikti įkelti savo modelius. Tokiu atveju galite nurodyti katalogą, kuriame yra visi susiję failai, įskaitant tokenizatoriaus parametrus, `config.json` failą su modelio parametrais, dvejetainius svorius ir kt.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objektas turi `encode` funkciją, kuri gali būti tiesiogiai naudojama tekstui užkoduoti:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Tada sukurkime iteratorius, kuriuos naudosime mokymo metu, kad pasiektume duomenis. Kadangi BERT naudoja savo kodavimo funkciją, mums reikės apibrėžti užpildymo funkciją, panašią į anksčiau apibrėžtą `padify`:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Mūsų atveju naudosime iš anksto apmokytą BERT modelį, vadinamą `bert-base-uncased`. Įkelkime modelį naudodami `BertForSequenceClassfication` paketą. Tai užtikrina, kad mūsų modelis jau turi reikiamą klasifikavimo architektūrą, įskaitant galutinį klasifikatorių. Pamatysite įspėjimo pranešimą, kad galutinio klasifikatoriaus svoriai nėra inicializuoti ir modelis reikalautų išankstinio apmokymo - tai visiškai normalu, nes būtent tai mes ir ketiname daryti!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Dabar esame pasiruošę pradėti mokymus! Kadangi BERT jau yra iš anksto apmokytas, norime pradėti nuo gana mažo mokymosi greičio, kad nepažeistume pradinių svorių.

Visą sunkų darbą atlieka `BertForSequenceClassification` modelis. Kai iškviečiame modelį su mokymo duomenimis, jis grąžina tiek nuostolį, tiek tinklo išvestį už pateiktą minibatch įvestį. Nuostolį naudojame parametrų optimizavimui (`loss.backward()` atlieka atgalinį skaičiavimą), o `out` naudojame mokymo tikslumui apskaičiuoti, lygindami gautas etiketes `labs` (apskaičiuotas naudojant `argmax`) su laukiamomis `labels`.

Norėdami kontroliuoti procesą, kaupiame nuostolį ir tikslumą per kelias iteracijas ir juos atspausdiname kas `report_freq` mokymo ciklų.

Šis mokymas greičiausiai užtruks gana ilgai, todėl ribojame iteracijų skaičių.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Galite pastebėti (ypač jei padidinsite iteracijų skaičių ir palauksite pakankamai ilgai), kad BERT klasifikacija suteikia mums gana gerą tikslumą! Taip yra todėl, kad BERT jau gana gerai supranta kalbos struktūrą, o mums tereikia pritaikyti galutinį klasifikatorių. Tačiau, kadangi BERT yra didelis modelis, visas mokymo procesas užtrunka ilgai ir reikalauja didelių skaičiavimo resursų! (GPU, ir geriausia, jei jų būtų daugiau nei vienas).

> **Pastaba:** Mūsų pavyzdyje naudojome vieną iš mažiausių iš anksto apmokytų BERT modelių. Yra didesnių modelių, kurie tikriausiai duotų geresnius rezultatus.


## Modelio veikimo vertinimas

Dabar galime įvertinti mūsų modelio veikimą testavimo duomenų rinkinyje. Vertinimo ciklas yra gana panašus į mokymo ciklą, tačiau nepamirškime perjungti modelio į vertinimo režimą, iškviečiant `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Pagrindinės mintys

Šiame skyriuje matėme, kaip lengva paimti iš anksto apmokytą kalbos modelį iš **transformers** bibliotekos ir pritaikyti jį mūsų teksto klasifikavimo užduočiai. Panašiai BERT modeliai gali būti naudojami objektų išskyrimui, klausimų atsakymui ir kitoms NLP užduotims.

Transformeriai yra dabartinė pažangiausia technologija NLP srityje, ir daugeliu atvejų tai turėtų būti pirmasis sprendimas, kurį pradedate testuoti, kai įgyvendinate individualius NLP sprendimus. Tačiau suprasti pagrindinius pasikartojančių neuroninių tinklų principus, aptartus šiame modulyje, yra itin svarbu, jei norite kurti pažangius neuroninius modelius.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipiame dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudotis profesionalių vertėjų paslaugomis. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus aiškinimus, kylančius dėl šio vertimo naudojimo.
